In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import os
import yaml
import json

In [2]:
json_file_path = '/kaggle/input/semis-od-coco-10/instances_val2017.json'

# Load the JSON file
with open(json_file_path, 'r') as f:
    data = json.load(f)
# Extract the annotations
annotations = data.get('images', [])

# Count the number of instances
num_instances = len(annotations)

print(f'Total number of instances: {num_instances}')


Total number of instances: 1232


In [3]:
file_path = '/kaggle/input/semis-od-coco-10/instances_val2017.json'

# Open and read the first 100 characters of the JSON file
with open(file_path, 'r') as file:
    first_100_chars = file.read(1000)

# Print the first 100 characters
print(first_100_chars)

{"images": [{"license": 4, "file_name": "000000149356.jpg", "coco_url": "http://images.cocodataset.org/train2017/000000149356.jpg", "height": 428, "width": 640, "date_captured": "2013-11-18 13:21:30", "flickr_url": "http://farm4.staticflickr.com/3349/3579581024_25d8e24a6c_z.jpg", "id": 149356}, {"license": 3, "file_name": "000000498400.jpg", "coco_url": "http://images.cocodataset.org/train2017/000000498400.jpg", "height": 427, "width": 640, "date_captured": "2013-11-17 17:10:19", "flickr_url": "http://farm8.staticflickr.com/7184/6892294630_44d92b1621_z.jpg", "id": 498400}, {"license": 2, "file_name": "000000498943.jpg", "coco_url": "http://images.cocodataset.org/train2017/000000498943.jpg", "height": 444, "width": 640, "date_captured": "2013-11-17 02:17:22", "flickr_url": "http://farm5.staticflickr.com/4049/4532685478_139af40701_z.jpg", "id": 498943}, {"license": 4, "file_name": "000000452205.jpg", "coco_url": "http://images.cocodataset.org/train2017/000000452205.jpg", "height": 428, "

In [4]:
import os
import json
import shutil
import random
from collections import defaultdict

# Define paths
coco_path = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
train_json = '/kaggle/input/semis-od-coco-10/instances_train2017_labeled.json'
val_json = '/kaggle/input/semis-od-coco-10/instances_val2017.json'
output_path = '/kaggle/working/yolo_dataset'

# Create output directories
os.makedirs(os.path.join(output_path, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels', 'val'), exist_ok=True)

# Load annotations
with open(train_json) as f:
    train_annotations = json.load(f)

with open(val_json) as f:
    val_annotations = json.load(f)

# Extract categories
categories = {category['id']: category['name'] for category in train_annotations['categories']}

# Function to convert COCO bbox to YOLO format
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2] / 2.0) * dw
    y = (box[1] + box[3] / 2.0) * dh
    w = box[2] * dw
    h = box[3] * dh
    return (x, y, w, h)

# Function to process annotations
def process_annotations(annotations, split):
    images = {image['id']: image for image in annotations['images']}
    seen_labels = defaultdict(set)  # Track seen labels for each image
    for ann in annotations['annotations']:
        image = images[ann['image_id']]
        bbox = convert_bbox((image['width'], image['height']), ann['bbox'])
        label = ann['category_id'] - 2   # YOLO class ids start at 0
        if (label < 0 or label > 7):
            print(label)
        bbox_tuple = (label, *bbox)  # Create a tuple of label and bbox for comparison

        if bbox_tuple not in seen_labels[image['id']]:  # Check for duplicates
            seen_labels[image['id']].add(bbox_tuple)
            # Write label file
            label_path = os.path.join(output_path, 'labels', split, f"{os.path.splitext(image['file_name'])[0]}.txt")
            with open(label_path, 'a') as f:
                f.write(f"{label} " + " ".join(map(str, bbox)) + '\n')

        # Copy image to split folder if not already copied
        target_image_path = os.path.join(output_path, 'images', split, image['file_name'])
        if not os.path.exists(target_image_path):
            shutil.copy(os.path.join(coco_path, image['file_name']), target_image_path)

# Process train and val annotations
process_annotations(train_annotations, 'train')
process_annotations(val_annotations, 'val')

# Create .yaml file
yaml_content = f"""
train: {os.path.join(output_path, 'images', 'train')}
val: {os.path.join(output_path, 'images', 'val')}

nc: {len(categories)}
names: {list(categories.values())}
"""

with open(os.path.join(output_path, 'dataset.yaml'), 'w') as f:
    f.write(yaml_content)


In [5]:
def read_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Example usage
file_path = '/kaggle/working/yolo_dataset/dataset.yaml'  # Replace with your file path
yaml_data = read_yaml(file_path)
print(yaml_data)

{'train': '/kaggle/working/yolo_dataset/images/train', 'val': '/kaggle/working/yolo_dataset/images/val', 'nc': 8, 'names': ['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat']}


In [6]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!wget https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt

Cloning into 'yolov9'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 668 (delta 224), reused 201 (delta 201), pack-reused 374
Receiving objects: 100% (668/668), 3.22 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (269/269), done.
/kaggle/working/yolov9
--2024-05-26 10:03:58--  https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/759338070/c8ca43f2-0d2d-4aa3-a074-426505bfbfb1?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20240526%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20240526T100358Z&X-Amz-Expires=300&X-Amz-Signature=58e80c8d3cd4cb92ee5a2666de49179b915038a783a51e8c2265d751f41d

### Trainning with multi-GPU

In [7]:
!torchrun \
    --nproc_per_node 2 \
    --master_port 9527 \
    train_dual.py \
    --workers 8 \
    --device 0,1 \
    --sync-bn \
    --batch 16 \
    --data /kaggle/working/yolo_dataset/dataset.yaml \
    --img 640 \
    --cfg models/detect/yolov9-c.yaml \
    --weights './yolov9-c.pt' \
    --name yolov9-c \
    --hyp hyp.scratch-high.yaml \
    --min-items 0 \
    --epochs 15 \
    --close-mosaic 15

[2024-05-26 10:04:02,333] torch.distributed.run: [WARNING] 
[2024-05-26 10:04:02,333] torch.distributed.run: [WARNING] *****************************************
[2024-05-26 10:04:02,333] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-05-26 10:04:02,333] torch.distributed.run: [WARNING] *****************************************
2024-05-26 10:04:13.287071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 10:04:13.287062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 10:04:13.287700: E external/local_xl

### Trainning with 1 GPU

In [8]:
# !python train_dual.py \
#     --workers 8 \
#     --device 0 \
#     --batch 8 \
#     --data /kaggle/working/yolo_dataset/dataset.yaml \
#     --img 640 \
#     --cfg models/detect/yolov9-c.yaml \
#     --weights './yolov9-c.pt' \
#     --name yolov9-c \
#     --hyp hyp.scratch-high.yaml \
#     --min-items 0 \
#     --epochs 5 \
#     --close-mosaic 15

In [9]:
!python val_dual.py \
    --img 640 \
    --batch 16 \
    --conf 0.001 \
    --iou 0.7 \
    --device 0 \
    --data /kaggle/working/yolo_dataset/dataset.yaml \
    --weights runs/train/yolov9-c3/weights/best.pt

val_dual: data=/kaggle/working/yolo_dataset/dataset.yaml, weights=['runs/train/yolov9-c3/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-89-g93f1a28 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "/kaggle/working/yolov9/val_dual.py", line 393, in <module>
    main(opt)
  File "/kaggle/working/yolov9/val_dual.py", line 366, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/kaggle/working/yolov9/val_dual.py", line 122, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/kaggle/working/y

In [10]:
!python3 detect_dual.py \
    --img 640 \
    --conf 0.6 \
    --weights runs/train/yolov9-c3/weights/best.pt \
    --data /kaggle/working/yolo_dataset/dataset.yaml \
    --source '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000461973.jpg'

detect_dual: weights=['runs/train/yolov9-c3/weights/best.pt'], source=/kaggle/input/coco-2017-dataset/coco2017/train2017/000000461973.jpg, data=/kaggle/working/yolo_dataset/dataset.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-89-g93f1a28 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "/kaggle/working/yolov9/detect_dual.py", line 232, in <module>
    main(opt)
  File "/kaggle/working/yolov9/detect_dual.py", line 227, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/kaggle/w